### The goal of this study is to reproduce the results of the paper “Semi-supervised Semantic Segmentation via Strong-weak Dual-branch Network".

### This file contains necessary definitions and functions for evaluating predictions made as outcomes of experiments with Single Branch and Dual Branch networks.

# 1.Data Preparation Steps

In [ ]:
import matplotlib.pyplot as plt # For plotting
import os
import cv2
import argparse
import numpy as np
import types
import copyreg
from multiprocessing import Pool
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!rm -rf VOCdevkit
# open up VOC dataset
!tar -xf /content/drive/MyDrive/VOC_Strong.tar

Resize predictions to their original image size

In [ ]:
from PIL import Image 
import PIL

def transformPredsToOriginalSize(pred_path, gt_path, image_list):
  image_list = [i.strip() for i in open(image_list) if not i.strip() == '']
  count =0
  for index, img_id in enumerate(image_list):
    count+=1       
    gt_img_path = os.path.join(gt_path, img_id + '.png')
    gt_img = Image.open(gt_img_path)
    width, height = gt_img.size
    
    pred_img_path = os.path.join(pred_path, img_id + '_pred.png')
    pred_img = Image.open(pred_img_path)
    im_resized = pred_img.resize((width, height))
    im_resized.save(pred_img_path)
    #print('Saved ', pred_img_path)

#2.Evaluate

##2.1. Confusion Matrix Class

In [ ]:
class ConfusionMatrix(object):

    def __init__(self, nclass, classes=None):
        self.nclass = nclass
        self.classes = classes
        self.M = np.zeros((nclass, nclass))

    def add(self, gt, pred):
        assert (np.max(pred) <= self.nclass)
        assert (len(gt) == len(pred))
        for i in range(len(gt)):
            if not gt[i] == 255:
                self.M[gt[i], pred[i]] += 1.0
            else:
                print("what'wrong", gt[i])

    def addM(self, matrix):
        assert (matrix.shape == self.M.shape)
        self.M += matrix

    def __str__(self):
        pass

    def jaccard(self):
        jaccard = 0.0
        jaccard_perclass = []
        for i in range(self.nclass):
            #if not self.M[i, i] == 0:
            jaccard_perclass.append(self.M[i, i] / (np.sum(self.M[i, :]) + np.sum(self.M[:, i]) - self.M[i, i]))

        return np.sum(jaccard_perclass) / len(jaccard_perclass), jaccard_perclass, self.M

    def generateM(self, item):
        gt, pred = item
        m = np.zeros((self.nclass, self.nclass))
        assert (len(gt) == len(pred))
        for i in range(len(gt)):
            if gt[i] < self.nclass:
                m[gt[i], pred[i]] += 1.0
        return m

##2.2. Utility Functions

In [ ]:

def _pickle_method(m):
    if m.im_self is None:
        return getattr, (m.im_class, m.im_func.func_name)
    else:
        return getattr, (m.im_self, m.im_func.func_name)


copyreg.pickle(types.MethodType, _pickle_method)

def semantic2mask(img):
    colors = np.array([(128, 0, 0), (0, 128,0 ), (128, 128, 0),
                  (0, 0, 128), (128, 0, 128), (0, 128, 128), (128, 128, 128),
                  (64, 0, 0), (192, 0, 0), (64, 128, 0), (192, 128, 0),
                  (64, 0, 128), (192, 0, 128), (64, 128, 128), (192, 128, 128),
                  (0, 64, 0), (128, 64, 0), (0, 192, 0), (128, 192, 0),
                  (0, 64, 128)]) # using palette for pascal voc
                  
    result = np.zeros((img.shape[0], img.shape[1]))
    for index, color in enumerate(colors):
        class_location = [img==color][0]
        loc = np.logical_and(np.logical_and(class_location[:,:,0], \
                                            class_location[:,:,1]), \
                             class_location[:,:,2])
        result[loc] = index+1
    return result.astype(dtype=int)

In [ ]:
def evaluate_all(pred_path, gt_path, image_list, save_name): 
  CATEGORY_LIST = ['background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair',
            'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant',
            'sheep', 'sofa', 'train', 'tvmonitor']
  class_num = 21

  print('Evaluation is executed.')
  iter = 0
  m_list = []
  data_list = []
  image_list = [i.strip() for i in open(image_list) if not i.strip() == '']
  count =0
  for index, img_id in enumerate(image_list):
    count+=1       
    gt_img_path = os.path.join(gt_path, img_id + '.png')
    gt = cv2.imread(gt_img_path)
    gt = cv2.cvtColor(gt, cv2.COLOR_BGR2RGB)
    gt = semantic2mask(gt)
  
    pred_img_path = os.path.join(pred_path, img_id + '_pred.png')
    pred = cv2.imread(pred_img_path)
    pred = cv2.cvtColor(pred, cv2.COLOR_BGR2RGB)
    pred = semantic2mask(pred)
               
    data_list.append([gt.flatten(), pred.flatten()])
        
  print('All images are loaded:', count)

  ConfM = ConfusionMatrix(class_num)

  f = ConfM.generateM
  pool = Pool()    
  m_list = pool.map(f, data_list)
  pool.close()
  pool.join()

  for m in m_list:
    ConfM.addM(m)

  aveJ, j_list, M = ConfM.jaccard()
  with open(save_name, 'w') as f:
    print('{0:12s}: {1:.4f}'.format('meanIOU', aveJ * 100))
    print('=' * 21)
    f.write('{0:12s}: {1:.4f}\n'.format('meanIOU', aveJ * 100))
    f.write('=' * 21)
    f.write('\n')
    for i, j in enumerate(j_list):
      print("{0:12s}: {1:.4f}".format(CATEGORY_LIST[i], j * 100))
      f.write("{0:12s}: {1:.4f}\n".format(CATEGORY_LIST[i], j * 100))

    f.write('Raw Result:\n')
    f.write('meanIOU: ' + str(aveJ) + '\n')
    f.write(str(j_list) + '\n')
    f.write(str(M) + '\n')
  
  return ConfM

##2.3.DSRG VOC Strong Dataset Evaluation

In [ ]:
!rm -rf VOCWeak
!mkdir VOCWeak
!cp -r /content/drive/MyDrive/VOCWeak/Results/pred VOCWeak/Results

pred_path = './VOCWeak/Results'
gt_path = './VOCdevkit/VOC2012/SegmentationClass'
image_list = './VOCdevkit/VOC2012/ImageSets/Segmentation/val.txt'
save_name = './VOCWeak/evaluation_out.txt'

evaluate_all(pred_path, gt_path, image_list, save_name)

Evaluation is executed.
All images are loaded: 1449
meanIOU     : 42.7052
background  : 77.5933
aeroplane   : 46.1848
bicycle     : 17.2869
bird        : 55.4742
boat        : 18.7217
bottle      : 35.9936
bus         : 59.7345
car         : 49.6030
cat         : 67.8699
chair       : 16.7156
cow         : 32.8326
diningtable : 21.9805
dog         : 60.5807
horse       : 39.6497
motorbike   : 53.1534
person      : 53.3878
pottedplant : 23.9378
sheep       : 57.1422
sofa        : 30.7602
train       : 38.2171
tvmonitor   : 39.9905


##2.4.Single Branch VOC Strong Dataset Evaluation

In [ ]:
# open up prediction archive 
!tar -xvf /content/drive/MyDrive/prediction_results_strongonly_2407/SingleBranchStrongData_pred.tar SingleBranchStrongData/Results/pred


In [ ]:
pred_path = './SingleBranchStrongData/Results/pred'
gt_path = './VOCdevkit/VOC2012/SegmentationClass'
image_list = './VOCdevkit/VOC2012/ImageSets/Segmentation/val.txt'
save_name = './SingleBranchStrongData/Results/evaluation_out.txt'

transformPredsToOriginalSize(pred_path, gt_path, image_list)
evaluate_all(pred_path, gt_path, image_list, save_name)

Evaluation is executed.
All images are loaded: 1449
meanIOU     : 22.3819
background  : 79.4983
aeroplane   : 27.9403
bicycle     : 0.0000
bird        : 15.9210
boat        : 1.2362
bottle      : 0.1984
bus         : 47.0878
car         : 33.9691
cat         : 40.5687
chair       : 0.0000
cow         : 10.1947
diningtable : 14.6867
dog         : 32.6843
horse       : 16.2989
motorbike   : 30.9792
person      : 31.9736
pottedplant : 0.0000
sheep       : 15.8128
sofa        : 2.0187
train       : 38.7185
tvmonitor   : 30.2321


##2.5.SingleBranch VOC Strong + SBD Weak Dataset Evaluation

In [ ]:
# open up prediction archive 
!tar -xf /content/drive/MyDrive/prediction_results_strongweak_2407/SingleBranchStrongWeakData_pred.tar SingleBranchStrongWeakData/Results/pred

In [ ]:
#!rm -rf SingleBranchStrongWeakData

# open up prediction archive 
!tar -xf /content/drive/MyDrive/prediction_results_strongweak_2407/SingleBranchStrongWeakData_pred.tar SingleBranchStrongWeakData/Results/pred

pred_path = './SingleBranchStrongWeakData/Results/pred'
gt_path = './VOCdevkit/VOC2012/SegmentationClass'
image_list = './VOCdevkit/VOC2012/ImageSets/Segmentation/val.txt'
save_name = './SingleBranchStrongWeakData/Results/evaluation_out.txt'


transformPredsToOriginalSize(pred_path, gt_path, image_list)
evaluate_all(pred_path, gt_path, image_list, save_name)

Evaluation is executed.
All images are loaded: 1449
meanIOU     : 28.0102
background  : 79.2818
aeroplane   : 36.2533
bicycle     : 0.0001
bird        : 21.4643
boat        : 21.7756
bottle      : 27.5242
bus         : 44.6864
car         : 40.8883
cat         : 40.8095
chair       : 1.7488
cow         : 22.5295
diningtable : 21.6110
dog         : 32.6826
horse       : 22.7688
motorbike   : 31.2457
person      : 28.2649
pottedplant : 9.9624
sheep       : 26.1169
sofa        : 11.4594
train       : 40.6032
tvmonitor   : 26.5373


##2.6.Dual Branch Evaluation




In [ ]:
!rm -rf DualBranch

# open up prediction archive 
!mkdir DualBranch
!mkdir DualBranch/Results
!mkdir DualBranch/Results/pred

!tar -xf /content/drive/MyDrive/DualBranch_pred.tar DualBranch/Results/pred

pred_path = './DualBranch/Results/pred'
gt_path = './VOCdevkit/VOC2012/SegmentationClass'
image_list = './VOCdevkit/VOC2012/ImageSets/Segmentation/val.txt'
save_name = './DualBranch/Results/evaluation_out.txt'


transformPredsToOriginalSize(pred_path, gt_path, image_list)
evaluate_all(pred_path, gt_path, image_list, save_name)

Evaluation is executed.
All images are loaded: 1449
meanIOU     : 3.3161
background  : 69.1397
aeroplane   : 0.0105
bicycle     : 0.0709
bird        : 0.0265
boat        : 0.0138
bottle      : 0.0016
bus         : 0.0096
car         : 0.0004
cat         : 0.0126
chair       : 0.0040
cow         : 0.0047
diningtable : 0.0006
dog         : 0.0041
horse       : 0.0028
motorbike   : 0.0000
person      : 0.0028
pottedplant : 0.0235
sheep       : 0.0037
sofa        : 0.0004
train       : 0.3011
tvmonitor   : 0.0044


In [ ]:
!tar -cf Results_single_branch_strong_data.tar SingleBranchStrongData/* 
!tar -cf Results_single_branch_strong_weak_data.tar SingleBranchStrongWeakData/* 